In [1]:
# Install dependencies
!pip install -qqq git+https://github.com/openai/whisper.git
!pip install -qqq transformers torchaudio
!pip install -qqq TTS
!sudo apt install ffmpeg
!apt-get install -y espeak

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.9 MB/s

In [2]:
import warnings
import whisper
import logging
import IPython.display as ipd

from transformers import MarianMTModel, MarianTokenizer
from TTS.api import TTS

warnings.filterwarnings('ignore')

2025-05-23 16:29:32.401512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748017772.723689      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748017772.814534      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
logging.getLogger("TTS").setLevel(logging.ERROR)
logging.getLogger("numba").setLevel(logging.WARNING)
logging.getLogger("transformers").setLevel(logging.ERROR)

In [4]:
# Load Whisper model
asr_model = whisper.load_model("base")

# This is referring to the `../b. Data/About Anger.mp3` file
audio_path = "/kaggle/input/about-anger-audio-file-mp3/About Anger.mp3"

# Display an audio player for the original audio file
ipd.Audio(audio_path)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 163MiB/s]


In [5]:
result = asr_model.transcribe(audio_path)
detected_lang = result["language"]  # This is the ISO 639-1 code, e.g., "en", "fr", "es"

print("🗣️ Detected Language:", detected_lang)

🗣️ Detected Language: en


In [6]:
# Transcribe
result = asr_model.transcribe(audio_path)
source_text = result["text"]
print("🗣️ Detected Speech:", source_text)

🗣️ Detected Speech:  We all know the feeling, the heart pounding, the hot rush of blood, the urge to shout. Yes, we are talking about anger, but what happens when that anger seems uncontrollable? You have seen or experienced it. A simple question ignites a fury. Why do certain words or actions provoke such intense reactions? Could it be that they touch a nerve, a raw, vulnerable spot? We dig deeper, trying to understand. Is it really about the question that was asked or is there something more? Sometimes it is not about what is happening around us, but what is happening within us?


In [7]:
tgt_lang = 'fr'

if detected_lang == tgt_lang:
    print("⚠️ Skipping translation: source and target languages are the same.")
    translated_text = source_text
else:
    model_name = f"Helsinki-NLP/opus-mt-{detected_lang}-{tgt_lang}"
    try:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        translator = MarianMTModel.from_pretrained(model_name)
    except Exception as e:
        print(f"❌ Could not load model for {model_name}. Error: {e}")
        translated_text = source_text  # Fallback: no translation

    inputs = tokenizer(source_text, return_tensors="pt", padding=True)
    translated_tokens = translator.generate(**inputs)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    print("Translated Text:", translated_text)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Translated Text: Oui, nous parlons de colère, mais que se passe-t-il quand cette colère semble incontrôlable? Vous l'avez vu ou expérimenté. Une question simple enflamme une furie. Pourquoi certains mots ou actions provoquent-ils des réactions aussi intenses? Est-ce qu'ils touchent un nerf, un point brut et vulnérable? Nous creuseons plus profondément, nous essayons de comprendre. Est-ce vraiment à propos de la question qui a été posée ou y a-t-il quelque chose de plus? Parfois, ce n'est pas à propos de ce qui se passe autour de nous, mais ce qui se passe en nous?


In [8]:
# Load the multilingual multi-speaker model
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", 
          progress_bar=False, 
          gpu=False)

# List available speakers
print("Available speakers:", tts.speakers)

# Use the first speaker for simplicity
speaker_id = tts.speakers[0]

# Generate speech
tts.tts_to_file(text=translated_text, 
                speaker=speaker_id, 
                language='fr-fr',
                file_path="About Anger - French.wav")

 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts
 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 

'About Anger - French.wav'

In [ ]:
# Output present in Outputs directory
# Display the generated audio
ipd.Audio("About Anger - French.wav")